Imports

In [10]:
import numpy as np
import cv2
import os
import glob

import matplotlib.pyplot as plt

from calibration_utils import *


from cv2 import aruco

Setting Source directory for camera calibration

In [11]:
root = './stereo_kalibracja_07_07_2023/'

data_path_left = root + '/cam2/'
data_path_right = root + '/cam4/'

img_ext = "*.png"

img_shape = (1280, 720)

Setting calibration values

In [12]:
board_shape = [11, 7]


# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

Preparing object points

In [13]:
dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)
parameters =  aruco.DetectorParameters()
arucoDetector = aruco.ArucoDetector(dictionary, parameters)


board = aruco.CharucoBoard(board_shape, 25, 19.0, dictionary)

Flag to display images while collecting data.

In [14]:
display_images = False

Processing Images

In [15]:

images_left = glob.glob(data_path_left + img_ext)
images_right = glob.glob(data_path_right + img_ext)
images_left.sort()
images_right.sort()

print(len(images_right))


allCornersLeft = []
allCornersRight = []
allIdsLeft = []
allIdsRight = []


corner_ids = cornerIds(board_shape)


for i, (left_path, right_path) in enumerate(zip(images_left, images_right)):
    print("[" + str((i+1)) + "/" + str(len(images_right)) + "]")
    head, tail = os.path.split(left_path)

    print(tail)
    
    
    bgr_left = cv2.imread(left_path, cv2.IMREAD_UNCHANGED)
    bgr_right = cv2.imread(right_path, cv2.IMREAD_UNCHANGED)

    rgb_left = cv2.cvtColor(bgr_left, cv2.COLOR_BGR2RGB)
    rgb_right = cv2.cvtColor(bgr_right, cv2.COLOR_BGR2RGB)

    gray_left = cv2.cvtColor(bgr_left, cv2.COLOR_BGR2GRAY)
    gray_right = cv2.cvtColor(bgr_right, cv2.COLOR_BGR2GRAY)
        
        
    corners_left,  ids_left,  rejectedCorners1 = arucoDetector.detectMarkers(gray_left)
    corners_right, ids_right, rejectedCorners2 = arucoDetector.detectMarkers(gray_right)

    if len(corners_left) > 16:
        for corner in corners_left:
            cv2.cornerSubPix(gray_left, corner,
                                winSize = (3,3),
                                zeroZone = (-1,-1),
                                criteria = criteria)
        if(display_images):
            frame_left_copy = aruco.drawDetectedMarkers(rgb_left, corners_left, ids_left)

    if len(corners_right) > 16:
        for corner in corners_right:
            cv2.cornerSubPix(gray_right, corner,
                                winSize = (3,3),
                                zeroZone = (-1,-1),
                                criteria = criteria)
        if(display_images):
            frame_right_copy = aruco.drawDetectedMarkers(rgb_right, corners_right, ids_right)
            
    
    if len(corners_left) > 16 and len(corners_right) > 16:
        res2_left  = cv2.aruco.interpolateCornersCharuco(corners_left, ids_left, gray_left, board)
        res2_right = cv2.aruco.interpolateCornersCharuco(corners_right, ids_right, gray_right, board)
       
           
        if res2_left[1] is not None and res2_right[1] is not None and res2_left[2] is not None and len(res2_left[1])>3 and max(ids_left) <= max(board.getIds()) and max(ids_left) <= max(board.getIds()):

            if is_slice_in_list(numpy.squeeze(ids_left).tolist(), corner_ids): # all left corners are detected
                if is_slice_in_list(numpy.squeeze(ids_right).tolist(), corner_ids): # all left corners are detected

                    allCornersLeft.append(res2_left[1])
                    allCornersRight.append(res2_right[1])

                    allIdsLeft.append(res2_left[2])
                    allIdsRight.append(res2_right[2])


    if display_images :
        f, axarr = plt.subplots(1,2)
        axarr[0].imshow(frame_left_copy)
        axarr[1].imshow(frame_right_copy)
        plt.show()
        

print("\nDetected " + str(len(images_left)) + " point pairs")

87
[1/87]
1.png
[2/87]
10.png


[3/87]
11.png
[4/87]
12.png
[5/87]
13.png
[6/87]
14.png
[7/87]
15.png
[8/87]
16.png
[9/87]
17.png
[10/87]
18.png
[11/87]
19.png
[12/87]
2.png
[13/87]
20.png
[14/87]
21.png
[15/87]
22.png
[16/87]
23.png
[17/87]
24.png
[18/87]
25.png
[19/87]
26.png
[20/87]
27.png
[21/87]
28.png
[22/87]
29.png
[23/87]
3.png
[24/87]
30.png
[25/87]
31.png
[26/87]
32.png
[27/87]
33.png
[28/87]
34.png
[29/87]
35.png
[30/87]
36.png
[31/87]
37.png
[32/87]
38.png
[33/87]
39.png
[34/87]
4.png
[35/87]
40.png
[36/87]
41.png
[37/87]
42.png
[38/87]
43.png
[39/87]
44.png
[40/87]
45.png
[41/87]
46.png
[42/87]
47.png
[43/87]
48.png
[44/87]
49.png
[45/87]
5.png
[46/87]
50.png
[47/87]
51.png
[48/87]
52.png
[49/87]
53.png
[50/87]
54.png
[51/87]
55.png
[52/87]
56.png
[53/87]
57.png
[54/87]
58.png
[55/87]
59.png
[56/87]
6.png
[57/87]
60.png
[58/87]
61.png
[59/87]
62.png
[60/87]
63.png
[61/87]
64.png
[62/87]
65.png
[63/87]
66.png
[64/87]
67.png
[65/87]
68.png
[66/87]
69.png
[67/87]
7.png
[68/87]
70.png
[69/87]
71.png
[70/87]


Preparing for stereo calibration

In [16]:
cameraMatrixInit = np.array([[ 1000.,    0., img_shape[0] / 2.],
                            [    0., 1000.,  img_shape[1] / 2.],
                            [    0.,    0.,           1.]])

distCoeffsInit = np.zeros((5,1))

flags = (cv2.CALIB_USE_INTRINSIC_GUESS + cv2.CALIB_RATIONAL_MODEL + cv2.CALIB_FIX_ASPECT_RATIO)
#flags = (cv2.CALIB_RATIONAL_MODEL)

In [17]:
print("Calculating ....")
                            
(ret_left, camera_matrix_left, distortion_coefficients_left,
_, _,
_, _,
_) = aruco.calibrateCameraCharucoExtended(
            charucoCorners=allCornersLeft,
            charucoIds=allIdsLeft,
            board=board,
            imageSize=img_shape,
            cameraMatrix=cameraMatrixInit.copy(),
            distCoeffs=distCoeffsInit.copy(),
            flags=flags,
            criteria=(cv2.TERM_CRITERIA_EPS & cv2.TERM_CRITERIA_COUNT, 10000, 1e-9))

(ret_right, camera_matrix_right, distortion_coefficients_right,
_, _,
_, _,
_) = aruco.calibrateCameraCharucoExtended(
            charucoCorners=allCornersRight,
            charucoIds=allIdsRight,
            board=board,
            imageSize=img_shape,
            cameraMatrix=cameraMatrixInit.copy(),
            distCoeffs=distCoeffsInit.copy(),
            flags=flags,
            criteria=(cv2.TERM_CRITERIA_EPS & cv2.TERM_CRITERIA_COUNT, 10000, 1e-9))

print("Done")

Calculating ....
Done


Collecting Shared Fetures

In [18]:
print("collectng shared corners ....")

shared_corners_l, shared_ids_l, shared_corners_r, \
    shared_ids_r = getSharedFetures(
        allCornersLeft, 
        allIdsLeft, 
        allCornersRight, 
        allIdsRight, 
        board)



imgPoints_l, objPoints_l = calculateImgPointsObjPoints(shared_ids_l, shared_corners_l, board)
imgPoints_r, objPoints_r = calculateImgPointsObjPoints(shared_ids_r, shared_corners_r, board)

print("Done")

collectng shared corners ....
Done


Stereo calibration

In [19]:


print("Calculating ....")

ret, M1, d1, M2, d2, R, T, E, F = cv2.stereoCalibrate(
    objPoints_l, 
    imgPoints_l,
    imgPoints_r,
    camera_matrix_left, distortion_coefficients_left, 
    camera_matrix_right, distortion_coefficients_right, 
    np.array(img_shape, np.int16),
    criteria=(cv2.TERM_CRITERIA_EPS & cv2.TERM_CRITERIA_COUNT, 10000, 1e-10), 
    flags=flags
)

print("Done")

Calculating ....
Done


Display Error

In [20]:
print("error: {}".format(ret))

error: 0.8012264888179846


Printing results

In [21]:
print('Intrinsic_mtx_1', M1)
print('Intrinsic_mtx_2', M2)

print('dist_1', d1)
print('dist_2', d2)

print('R', R)
print('T', T)
print('E', E)
print('F', F)

print('Baseline', np.linalg.norm(T) * 0.1)

Intrinsic_mtx_1 [[721.15383959   0.         617.99175832]
 [  0.         721.15383959 358.73452559]
 [  0.           0.           1.        ]]
Intrinsic_mtx_2 [[727.77467307   0.         618.10836308]
 [  0.         727.77467307 359.80047653]
 [  0.           0.           1.        ]]
dist_1 [[ 5.47140688e+00]
 [-3.74005498e+00]
 [ 2.96701789e-03]
 [-2.26053742e-03]
 [ 4.30713610e+00]
 [ 5.26941301e+00]
 [-3.74150551e+00]
 [ 4.51170346e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]
dist_2 [[ 3.40114619e+00]
 [-8.92630926e+00]
 [ 3.81056610e-03]
 [ 1.97130110e-04]
 [ 6.13298568e+00]
 [ 3.17823730e+00]
 [-8.48482211e+00]
 [ 5.90119992e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]
R [[ 9.99961388e-01  4.45395357e-03 -7.57531990e-03]
 [-4.45379201e-03  9.99990081e-01  3.81967850e-05]
 [ 7.57541488e-03 -4.45641088e-06  9.99971306e-01]]
T [[123.43

Saving results

In [22]:
np.savetxt('Intrinsic_mtx_1.txt', M1)
np.savetxt('Intrinsic_mtx_2.txt', M2)

np.savetxt('dist_1.txt', d1)
np.savetxt('dist_2.txt', d2)

np.savetxt('R.txt', R)
np.savetxt('T.txt', T)